# Data Prep, Scaling, Train-Test-Validate, and Pipelines.

Today we will follow **Chapter 2 of our book: Hands-on Machine Learning**. We will work our way through the overview of a complete machine learning project. Our goals include:

Understanding the importance of: 
* data cleaning, prep, and visualization
* separating the data into training, testing, and validating sets
* normalizing or scaling the data
* how to set up a pipeline to do much of this work (advanced)

We will also learn about the Python package sklearn:
https://scikit-learn.org/stable/getting_started.html

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1u8T1tUeCCnNfMutsMKco6oUSs7mzuDvq?usp=sharing)


### Preliminary Cell

It is a good idea to just copy and past your preliminary cell into each new notebook once you have one that works for you!

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import pandas as pd

# Start a list of useful packages
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression



from pandas.plotting import scatter_matrix

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

<span style="color:blue">

# 1. Get the data - California Housing Prices 
</span>

### GOAL - to predict median home price given other information.


* In this class we will provide data sets for you to investigate. 

* There are lots of interesting machine learning data sets that you can explore here: https://archive.ics.uci.edu/ml/index.php

* Sometimes the hardest part of an ML project is finding good, reliable, data and understanding the data that you have.



In [ ]:
# If you did not want to use Pandas you would do this:

# import os
# import tarfile
# import urllib

# DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
# HOUSING_PATH = os.path.join("datasets", "housing")
# HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
#     if not os.path.isdir(housing_path):
#         os.makedirs(housing_path)
#     tgz_path = os.path.join(housing_path, "housing.tgz")
#     urllib.request.urlretrieve(housing_url, tgz_path)
#     housing_tgz = tarfile.open(tgz_path)
#     housing_tgz.extractall(path=housing_path)
#     housing_tgz.close()
    
# fetch_housing_data()

In [ ]:
# THE DATA FOR TODAY

# Exact location of the .csv file
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.csv"

# Load it using pandas
housing = pd.read_csv(HOUSING_URL)
# You could write a nice function to do this!

# Start looking at what we have
housing.head()

<span style="color:blue">
    
# 2. Look at the data
</span>

Answer the following questions about the data:
* What type of data is in each column?
* How many entries are in our data set?
* What are all of the column names?
* What are the max, min, and mean of each column?
* What are the counts for each item in the ocean_proximity column?
* Are there any NaNs in the data? If so, look 

#### DON'T

#### PEEK

#### AT

#### THE

#### SOLUTION!

### Answers:

In [ ]:
print('Data in each column:')
print(housing.info())

print('--------------------------------------')

print('Entries in our data set:')
print(housing.shape[0])

print('--------------------------------------')

print('Column names:')
print(housing.columns)

print('--------------------------------------')

print("Data about max, min, mean, and more:")
display(housing.describe())

print('--------------------------------------')

print('Counts for itmes in ocean_proximity column:')
print(housing["ocean_proximity"].value_counts())

print('--------------------------------------')

print('Check for NaNs:')
print(housing.isna().sum())

print('--------------------------------------')

print('Rows that contain NaNs')
display(housing[housing['total_bedrooms'].isna()])

<span style="color:blue">

# 3.  Do some preliminary data visualization

</span>

In [ ]:
# Easiest way is to plot the data in matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

### Check assumptions often!

* Trustworthy source?
* Any unusual information?
* Could the data be biased or could there be errors?
* Any unexpected values or units?

<span style="color:blue">

# 4. IMPORTANT: Test - Train Split

</span>

* At this point, once you trust and understand your data, you want to set aside some of your data for testing your model. 
* You will not "look" at this data again until your machine learning algorithm is read to test.

### Training set: 
Usually about 80% of your data, to be used in building/training your model.

### Testing set:
Usually about 20% of your data, to be used in testing how well your model makes predictions on new data. This data could be split into testing and validation sets if you have a lot of parameter tuning to do - more about this later in the class.


### SKLEARN
One of the best basic machine learning python packages! 

Many of the algorithms that we will learn in class are part of the sklearn library. Remember **It is important to understading the underlying mathematics** if you want to do good machine learning. Please avoid falling into the black box machine learning mentality!


In [ ]:
# to make this notebook's output identical at every run
# you would not need this for individual projects - just for in class demos
np.random.seed(42)

#### Random samples:

Spliting your data set randomly works well if you have a huge dataset or if you data set has very little bias. You are assuming that you will not introduce sampling bais by taking a random sample



In [ ]:
# RANDOM
# Use test_train_split to get two different datasets each randomly sampled:
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
train_set.describe()

In [ ]:
test_set.describe()

In [ ]:
# Notice how we could get a training set with almost no median incomes at the higher end of the tail.
housing["median_income"].hist()

#### Stratified Samples:

You want to make sure your data samples match the overall population. Example: Your full dataset contains pictures of 700 cats and 300 dogs and you want to train a model that identifies cats from dogs. You could randomly sample a training set that contains 700 cats and 100 dogs and a test set that contains 200 dogs. Not good. Instead, stratified samples make sure that certain categories in the data are represented at the correct percentages.

In [ ]:
# Make a new column in our Data Frame that categorizes the median income into levels with labels
# 1 - ranges from 0-1.5
# 2 - ranges from 1.5-3
# 3 - ranges from 3-4.5
# 4 - ranges from 4.5-6
# 5 - anything higher than 6 (to np.inf)

# Here we are chosing the categories - these are part of our model assumptions.

housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
# How much data falls into each bin?
housing["income_cat"].value_counts()

In [ ]:
# STRATIFIED
# Here we split the data: 

# First we use sklearn to find the indices of how we would split our data for each column
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# n_splits=1 is 1-fold

# Get the indicees that give us the stratified random split out of the split object
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
# Here we can see the different proportions

def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()
compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

compare_props

In [ ]:
# Now delete the extra data column that we added - it's not really data we need later
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

<span style="color:blue">
    
# 5. Explore and visualize the TRAINING data to gain insights

</span>

* Here we are now only using the training set so that we keep the test set "out of sight" of our model
* There are lots of different ways to visualize the data. 
* Our goal is to try to decide what kind of a machine learning model might work best and what variables are the best predictors

Keep your question in mind!

In [ ]:
# Get a clean copy of the training set
housing = strat_train_set.copy()

### Scatter Plots

In [ ]:
# A simple scatter plot
housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7))

In [ ]:
# A scater plot that causes high density points to be darker
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1, figsize=(10,7))

In [ ]:
# housing.plot? # You can explore these functions!!

In [ ]:
# A super fancy plot that:
# -- sets the radius to represent population
# -- sets a color map to housing proces
# -- sets high density points to be darker

housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=housing["population"]/100, label="population", figsize=(10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)
plt.legend()

# Here s=size and c=color

#### Make some observations:

* Looking at the data what trends do you see? 
* It's a good idea to write a paragraph or two about what you see in the data at this point!

### Exploring Correlations

In [ ]:
# Create a correlation matrix
corr_matrix = housing.corr()

In [ ]:
# Look to see which things are correlated with the variable of interest
corr_matrix["median_house_value"].sort_values(ascending=False)

#### What can corrrelation tell us?

In [ ]:
# Create bar graphs and scatter plots for the variables you are interested in.

# Variables of interest
attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]

# use the Pandas scatter_matrix to make plots)
scatter_matrix(housing[attributes], figsize=(12, 8))

#### Which of these plots helps answer our question?

In [ ]:
# Choose some of the highly correlated ones to zoom in on
housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1)
plt.axis([0, 16, 0, 550000])

### Explore some new attributes

In [ ]:
# Create the new attributes
# These are now columns in your training set
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
housing.plot(kind="scatter", x="rooms_per_household", y="median_house_value",
             alpha=0.2)
plt.axis([0, 5, 0, 520000])
plt.show()

In [ ]:
housing.plot(kind="scatter", x="bedrooms_per_room", y="median_house_value",
             alpha=0.2)
plt.axis([0, 1, 0, 520000])
plt.show()

In [ ]:
housing.describe()

<span style="color:blue">

# 6. Prepare the data for Machine Learning algorithms - preprocessing

</span>

* Only now are we ready to start preparing for machine learning. 
* Notice how much data prep and exploration we have already done and there is still more!

In [ ]:
# Get a clean version of the training data
# Just in case you did something weird during exploration

# FEATURES or INPUT PARAMETERS
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
# LABELS or OUTPUT PARAMETERS
housing_labels = strat_train_set["median_house_value"].copy()

# We want to use the input parameters or features to predict median_house_values or labels.

### Consider what to do with "bad" data or NaNs.

Sometimes this is done at the top when you are looking at your data. That is okay too.

In [ ]:
# Find any rows in the training data that contain a NaN
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

#### What should we do with these NaNs?

We have options:
1. Just drop the rows that have a NaN - this must be done to both the Features and the Labels.
2. Just get rid of the column that has NaNs and do let it be an input.
3. Find some value to use to fill the NaNs - the median, max, min, etc

In [ ]:
# sample_incomplete_rows.dropna(subset=["total_bedrooms"])    # option 1

In [ ]:
# sample_incomplete_rows.drop("total_bedrooms", axis=1)       # option 2

In [ ]:
median = housing["total_bedrooms"].median() 
housing["total_bedrooms"].fillna(median, inplace=True) # option 3

In [ ]:
# Check that it worked
housing[housing.isnull().any(axis=1)].head()

#### Be Careful

1. Always calculate medians, max, etc based only on information from the training set. **Your model is not allowed to see the testing set!**
2. If you remove rows from the features you must remove those rows from the labels.
2. Remember what you did here! 
3. when it comes time to use the testing data, you will have to repete ALL of these steps exactly the same way.

Eg. We would fillna in our test set with the SAME median found from our training set!

### How do we deal with categorical (non numerical) data?

Often our data sets contian data that is not numerical. In this dataset we have the input ocean_proximity. There are a few ways to deal with this:

1. **Ordinal Encoding:** Create a single column in your dataset that numbers the categories. For ocean proximity we will have 0, 1, 2, 3, 4 to represent the five categories.

2. **One Hot Encoding:** Create a column of arays that identify the category by zeros and ones. The first category will be [1,0,0,0,0] the next [0,1,0,0,0] and so on.

In [ ]:
# Look at the labels
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

#### Ordinal Encoder - sklearn

In [ ]:
# Define the encoder
ordinal_encoder = OrdinalEncoder()

# Send the categorical data through the encoder
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

# This produces an aray of values that we could add as a column in our Data Frame
housing_cat_encoded[:10]

In [ ]:
# The categories are saved so that later you could reincode
print(ordinal_encoder.categories_)

# You can undo the encoding 
ordinal_encoder.inverse_transform(housing_cat_encoded[:10])

#### One hot encoding - sklearn

In [ ]:
# Define the encoder
cat_encoder = OneHotEncoder(sparse=False)

# Send the categorical data through the encoder
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

# Try to print the information
housing_cat_1hot

By default, the `OneHotEncoder` class returns a sparse array, which saves memory by not saving zeros. This makes sense for one hot encoding, but is not great for users who want the data. This is why we set `sparse = False`

In [ ]:
# We still save the categories
cat_encoder.categories_

# We can still invert
cat_encoder.inverse_transform(housing_cat_1hot)

#### Label Encoder - sklearn - another way

With everything you will see in this class there are **multiple ways to do the same thing**. Here is a fun package called LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Define th encoder
le=LabelEncoder()
# Fit the classes
le.fit(housing["ocean_proximity"])
list(le.classes_)

In [ ]:
# Encode
le.transform(housing["ocean_proximity"])

### OPTIONAL Add new features 

Remember if you want to use anything in your model, you would need to add it as a column to your features dataframe

In [ ]:
# Just as an Example:

### If I watned to I could add some attributes and encodings to the dataset:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]
housing["one_hot_locations"]=housing_cat_1hot.tolist()

housing.head()

In [ ]:
# Check for any remaining NaNs
display(housing[housing.isnull().any(axis=1)].head())

#### What have we done here:
1. We got a fresh copy of the training data
2. We broke it into Features (Inputs) and Labels (Outputs)
3. We delt with NaNs in our data
4. We added new features: new attributes and label encodings.


#### If you are a more advanced programmer you cade defined your own class function that will fit and transform the data adding new attributes automatically! (see code at the bottom of this notebook)

But this is not expected in this class. See the book for more infomation!

<span style="color:blue">

# 7. Feature Scaling

</span>

You pretty much **ALWAYS DO THIS**!! This is importnat with numerical data. With categorical not so much, you can usually leave those columns alone.

In most cases you will want to make sure that each of your features (x) are normalized or scaled. What this means is that we want the data to have a few characteristics:

- Each of the features has approximately the same scale
- Most of your data falls between -1 and 1
- This is not unique and there are lots of ways to do this... here are two

- This helps numerical optimization methods converge faster and helps avoid over/under flow errors
- Much more important in problems with many features!

### Min Mix Normalization

The simplest method 

The general formula for a min-max of [0, 1] is given as:

$$  x'={\frac {x-{\text{min}}(x)}{{\text{max}}(x)-{\text{min}}(x)}} $$

This scales your data to 0 < x' < 1

### Mean Normalization

Simple Mean normalization

$$ x'={\frac {x-{\text{average}}(x)}{{\text{max}}(x)-{\text{min}}(x)}} $$


### Standardization

Similar to above just divide by the standard deviation. Ensures your data has zero mean and unit variance.

$$ x'={\frac {x-{\text{average}}(x)}{{\text{std}} (x)}} $$

In [ ]:
# Standard Scalar from sklearn
scalar = StandardScaler()

# This works only with numerical data: 
# housing_scaled = scalar.fit_transform(housing) # Gives an error
# So we drop the categorical collumns to proceed
housing_num = housing.drop(columns=["ocean_proximity","one_hot_locations"])
housing_num.head()


In [ ]:
housing_scaled = scalar.fit_transform(housing_num)
housing_scaled

In [ ]:
housing_scaled.shape

#### Questions:

* How many FEATURES do we have?
* How many data points do we have?

In [ ]:
# We could look at the data as a data frame just to confirm all the numbers look right
checkdf = pd.DataFrame(housing_scaled)
checkdf

<span style="color:blue">

# 8. Select and train a model 

</span>

* The rest of our class will be concerned with developing different machine learning models. 
* You are expected to do all of the data exploration, splitting, and preparaion for each project you do!

**Here is a simple example of doing machine learning on this data**

#### Training

Since we did all our data prep above, training is simple

In [ ]:
# We will choose simple linear regression - you will learn abou this soon!

# Define the machine learning model
lin_reg = LinearRegression()
# Learn....
lin_reg.fit(housing_scaled, housing_labels)

#### Testing

Remember all the data prep we did with the training data. We have to redo this on the testing data!

**Get a copy the data**

In [ ]:
# Our model now "knows" the parameters that best fit the training data
# Lets test the model on the testing data

# We have to redo all the preprocessing steps we did to the training data

# Get the test data
mini_test_data = strat_test_set

# Break into Features and Labels
# FEATURES or INPUT PARAMETERS
housing_test = mini_test_data.drop("median_house_value", axis=1) # drop labels for training set
# LABELS or OUTPUT PARAMETERS
housing_labels_test =  mini_test_data["median_house_value"].copy()


**Check for NaNs and apply the same opperation**

In [ ]:
# Check for NaNs
housing_test[housing_test.isnull().any(axis=1)].head()

In [ ]:
# Same NaN treatment - using the variable median from the training data
housing_test["total_bedrooms"].fillna(median, inplace=True)

In [ ]:
# Check for NaNs again
housing_test[housing_test.isnull().any(axis=1)].head()

**Add any additional features that you used in training**

Remember that even though we did some operations on the categorical data, in the end we dropped the categorical data from our features before we scaled the data.

In [ ]:
# Add new features or atributes that were used in training
housing_test["rooms_per_household"] = housing_test["total_rooms"]/housing_test["households"]
housing_test["bedrooms_per_room"] = housing_test["total_bedrooms"]/housing_test["total_rooms"]
housing_test["population_per_household"]=housing_test["population"]/housing_test["households"]

housing_test.head()

In [ ]:
# Drop non-numerical columns
housing_num_test = housing_test.drop(columns=["ocean_proximity"])


**Apply Standardization**

You can reuse the scalar oblect that you defined above

* Training use `fit_transform`
* Testing use `transform` only

In [ ]:
# Scale the input parameters
housing_test_scaled = scalar.transform(housing_num_test)

**Double check the shapes!**

Your scaled features from the testing data should match the number of scaled features you had in the training data.

In [ ]:
# Check shapes
print("TESTING DATA")
print(housing_test_scaled.shape)
print("TRAINING DATA")
print(housing_scaled.shape)

But wait! These don't match exactly! **WHY?**

**Now send the data though the model to get predictions**

In [ ]:
# Now feed the test features into your model
model_predictions = lin_reg.predict(housing_test_scaled)
model_predictions

* What are these numbers? 
* What should our model predict? 
* How well did it do?

In [ ]:
# We could just look at the difference
percent_diff = np.abs((model_predictions-housing_labels_test))/housing_labels_test*100
df_test = pd.DataFrame({"model":model_predictions,"real data":housing_labels_test,"percent": percent_diff})
df_test.head(30)

In [ ]:
# We will learn about many metrics for measuring the goodness of a model
from sklearn.metrics import mean_squared_error

lin_mse = mean_squared_error(housing_labels_test, model_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

**We are clearly getting a lot of these wrong by quite a bit**

<span style="color:blue">

# 9. Fine-tune your model or choose a different model or maybe reconsider your data or...

</span>

Machine learning and mathematical modeling is a creative process. You have to use your instincts and make hard decisions. Sometimes your model just works great! Most of the time it does not and you go back and rethink your steps.

**Some good advice**
* keep track of your assumptions and decisions
* comment your code
* be really cynical about your results

# EXTRA - More advanced programming

You will notice that we ended up doing a lot of things twice. Once to our training data and then again to our testing data. 

Often it is nice to define a special function that will do all of these things for you. 

Sklearn has something called a **pipeline** that will do this for us.

### Pipeline for training data prep

In [ ]:
# Get a fresh copy of the data
# FEATURES or INPUT PARAMETERS
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
# Drop categorical or string columns
housing_num = housing.drop(columns=["ocean_proximity"])

# LABELS or OUTPUT PARAMETERS
housing_labels = strat_train_set["median_house_value"].copy()


# To automatically deal with NaNs use SimpleImputer
from sklearn.impute import SimpleImputer

# To automatically add new attributes, define a class that does this
from sklearn.base import BaseEstimator, TransformerMixin
# The class as writen below needs to know the column index
# eg total rooms is in column 4 so to python this is index 3
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        # add new attributes
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Pipeline allows you to do multiple operations one after another
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), #First we get rid of nans
        ('attribs_adder', CombinedAttributesAdder()), #Then we add the new data
        ('std_scaler', StandardScaler()), #Then we use standard scalar
    ])



**Preprocessing the data by sending it through the pipeline**

`fit_transform`

In [ ]:
# Once you defined all the thigns above
# Then you can do all the stuff to the data in one line
housing_num_tr = num_pipeline.fit_transform(housing_num)
housing_num_tr

**Do the same steps to the testing data by sending it through the pipeline**

`transform`

In [ ]:
# Get a fresh copy of the testing data
# FEATURES or INPUT PARAMETERS
housing_test = strat_test_set.drop("median_house_value", axis=1) # drop labels for training set
# Drop categorical or string columns
housing_num_test = housing_test.drop(columns=["ocean_proximity"])

# Do all the stuff again to the testing data
housing_num_test_tr = num_pipeline.transform(housing_num_test)
housing_num_test_tr

### Pipeline that includes the linear regression model

In [ ]:
# We could even add the machine learning model to the pipeline
# This still uses the classes we defined above

# Get a fresh copy of the data
# FEATURES or INPUT PARAMETERS
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
# Drop categorical or string columns
housing_num = housing.drop(columns=["ocean_proximity"])

# LABELS or OUTPUT PARAMETERS
housing_labels = strat_train_set["median_house_value"].copy()

full_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), #First we get rid of nans
        ('attribs_adder', CombinedAttributesAdder()), #Then we add the new data
        ('std_scaler', StandardScaler()),#Then we use standard scalar
        ('lin_reg', LinearRegression()),#Then do the model
    ])


**Training**

In [ ]:
full_pipeline.fit(housing_num,housing_labels)

**Testing**

In [ ]:
# Get a fresh copy of the testing data
# FEATURES or INPUT PARAMETERS
housing_test = strat_test_set.drop("median_house_value", axis=1) # drop labels for training set
# Drop categorical or string columns
housing_num_test = housing_test.drop(columns=["ocean_proximity"])

# Get the predictions in one line
full_pipeline.predict(housing_num_test)

### Advanced programming - scaling/normalizing with categorical data

See the book chapter 2 for more information

In [ ]:
### There are also methods for writing classes and function 
### for dealing with both categorical and numerical data

from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

from sklearn.base import BaseEstimator, TransformerMixin

# Create a class to select numerical or categorical columns 
class OldDataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
    num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

old_num_pipeline = Pipeline([
        ('selector', OldDataFrameSelector(num_attribs)),
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

old_cat_pipeline = Pipeline([
        ('selector', OldDataFrameSelector(cat_attribs)),
        ('cat_encoder', OneHotEncoder(sparse=False)),
    ])

from sklearn.pipeline import FeatureUnion

old_full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", old_num_pipeline),
        ("cat_pipeline", old_cat_pipeline),
    ])